## Downloading NEXRAD data
Scans are an available open data source: https://www.ncdc.noaa.gov/data-access/radar-data/noaa-big-data-project
They are stored in AWS warehousing by year, month, date, and time of scan for each radar location. 

Documentation: https://docs.opendata.aws/noaa-nexrad/readme.html
code adopted from https://www1.ncdc.noaa.gov/pub/data/radar/bdp/scripts/

Website to get sunrise time: https://www.esrl.noaa.gov/gmd/grad/solcalc/sunrise.html

In [45]:
import sys
import json
import datetime
import urllib.request
import pandas as pd
import nexradaws

sys.path.append('../')
from envir import config

##### Define Functions

In [46]:
def getSunriseTime(dateString):
    date = dateString.replace("-", "/")
    # get sunrise time
    sundoc = urllib.request.urlopen('https://api.sunrise-sunset.org/json?lat=40.730959&lng=-73.9987294&date='+date)
    sundoc = json.loads(sundoc.read())
    sunrise = sundoc['results']['sunrise'][:-3].split(':')
    sunrise = ''.join(sunrise)
    return(sunrise)

In [47]:
def getSunsetTime(dateString):
    date = dateString.replace("-", "/")
    # get sunrise time
    sundoc = urllib.request.urlopen('https://api.sunrise-sunset.org/json?lat=40.730959&lng=-73.9987294&date='+date)
    sundoc = json.loads(sundoc.read())
    sunset = sundoc['results']['sunset'][:-3].split(':')
    sunset = ''.join(sunset)
    return(sunset)

In [48]:
def getSunriseScans(dateString, radar, fileLocation):
    year = int(dateString.split("-")[0])
    month = int(dateString.split("-")[1])
    day = int(dateString.split("-")[2])
    sunrise = getSunriseTime(dateString)
    hour = int(sunrise[:2])
    minute = int(sunrise[2:4])
    sec = int(sunrise[4:])
    
    start = datetime.datetime(year, month, day, hour, minute, sec)
    end = start + datetime.timedelta(minutes = 10)
    
    conn = nexradaws.NexradAwsInterface()
    availscans = conn.get_avail_scans_in_range(start, end, radar)
    print("There are {numscans} NEXRAD files available for {month}/{day}/{year} for the {radar} radar.\n".format(numscans = len(availscans),
                                                                                                                 month = month,
                                                                                                                 day = day,
                                                                                                                 year = year, 
                                                                                                                 radar = radar))
    print(availscans[0:4])
    results = conn.download(availscans, fileLocation)

In [49]:
def getSunsetScans(dateString, radar, fileLocation):
    year = int(dateString.split("-")[0])
    month = int(dateString.split("-")[1])
    day = int(dateString.split("-")[2])
    sunset = getSunsetTime(dateString)
    hour = int(sunset[:2])
    minute = int(sunset[2:4])
    sec = int(sunset[4:])
    
    start = datetime.datetime(year, month, day, hour, minute, sec)
    end = start + datetime.timedelta(minutes = 10)
    
    conn = nexradaws.NexradAwsInterface()
    availscans = conn.get_avail_scans_in_range(start, end, radar)
    print("There are {numscans} NEXRAD files available for {month}/{day}/{year} for the {radar} radar.\n".format(numscans = len(availscans),
                                                                                                                 month = month,
                                                                                                                 day = day,
                                                                                                                 year = year, 
                                                                                                                 radar = radar))
    print(availscans[0:4])
    results = conn.download(availscans, fileLocation)

In [50]:
def getScans(dateString, radar, fileLocation):
    year = dateString.split("-")[0]
    month = dateString.split("-")[1]
    day = dateString.split("-")[2]
    
    conn = nexradaws.NexradAwsInterface()
    availscans = conn.get_avail_scans(year, month, day, radar)
    print("There are {numscans} NEXRAD files available for {month}/{day}/{year} for the {radar} radar.\n".format(numscans = len(availscans),
                                                                                                                 month = month,
                                                                                                                 day = day,
                                                                                                                 year = year, 
                                                                                                                 radar = radar))
    print(availscans[0:4])
    results = conn.download(availscans, fileLocation)

#### download scans for a range of dates

In [51]:
date_range = pd.date_range(start="2019/03/01", end="2019/03/30").astype(str)

In [40]:
date_range

Index(['2019-03-01', '2019-03-02', '2019-03-03', '2019-03-04', '2019-03-05',
       '2019-03-06', '2019-03-07', '2019-03-08', '2019-03-09', '2019-03-10',
       '2019-03-11', '2019-03-12', '2019-03-13', '2019-03-14', '2019-03-15',
       '2019-03-16', '2019-03-17', '2019-03-18', '2019-03-19', '2019-03-20',
       '2019-03-21', '2019-03-22', '2019-03-23', '2019-03-24', '2019-03-25',
       '2019-03-26', '2019-03-27', '2019-03-28', '2019-03-29', '2019-03-30'],
      dtype='object')

In [43]:
#for d in date_range:
#    getSunriseScans(d, 'KOKX', config.dataFol+'nexrad_downloads')

There are 2 NEXRAD files available for 3/1/2019 for the KOKX radar.

[<AwsNexradFile object - 2019/03/01/KOKX/KOKX20190301_113144_V06>, <AwsNexradFile object - 2019/03/01/KOKX/KOKX20190301_113739_V06>]
Downloaded KOKX20190301_113144_V06
Downloaded KOKX20190301_113739_V06
2 out of 2 files downloaded...0 errors
There are 2 NEXRAD files available for 3/2/2019 for the KOKX radar.

[<AwsNexradFile object - 2019/03/02/KOKX/KOKX20190302_112948_V06>, <AwsNexradFile object - 2019/03/02/KOKX/KOKX20190302_113658_V06>]
Downloaded KOKX20190302_112948_V06
Downloaded KOKX20190302_113658_V06
2 out of 2 files downloaded...0 errors
There are 2 NEXRAD files available for 3/3/2019 for the KOKX radar.

[<AwsNexradFile object - 2019/03/03/KOKX/KOKX20190303_112630_V06>, <AwsNexradFile object - 2019/03/03/KOKX/KOKX20190303_113455_V06>]
Downloaded KOKX20190303_112630_V06
Downloaded KOKX20190303_113455_V06
2 out of 2 files downloaded...0 errors
There are 2 NEXRAD files available for 3/4/2019 for the KOKX radar.

In [53]:
getScans('2017-05-17', 'KOKX', config.dataFol+'nexrad_downloads')

There are 183 NEXRAD files available for 05/17/2017 for the KOKX radar.

[<AwsNexradFile object - 2017/05/17/KOKX/KOKX20170517_000755_V06>, <AwsNexradFile object - 2017/05/17/KOKX/KOKX20170517_001736_V06>, <AwsNexradFile object - 2017/05/17/KOKX/KOKX20170517_002717_V06>, <AwsNexradFile object - 2017/05/17/KOKX/KOKX20170517_003910_V06>]
Downloaded KOKX20170517_001736_V06
Downloaded KOKX20170517_000755_V06
Downloaded KOKX20170517_003910_V06
Downloaded KOKX20170517_005834_V06
Downloaded KOKX20170517_002717_V06
Downloaded KOKX20170517_004851_V06
Downloaded KOKX20170517_010815_V06
Downloaded KOKX20170517_011758_V06
Downloaded KOKX20170517_012741_V06
Downloaded KOKX20170517_013724_V06
Downloaded KOKX20170517_014705_V06
Downloaded KOKX20170517_021611_V06
Downloaded KOKX20170517_015647_V06
Downloaded KOKX20170517_020629_V06
Downloaded KOKX20170517_022553_V06
Downloaded KOKX20170517_023536_V06
Downloaded KOKX20170517_024518_V06
Downloaded KOKX20170517_025837_V06
Downloaded KOKX20170517_025501_V